<h1 align=center><font size = 5>Final Project</font></h1>

## Idea:
-I have an Italian Investor who wants to open an Italian restaurant in New York and he is looking for the best neighborhood for his restaurant<br/>
-The idea is to find a neighborhood who doesn't have an  Italian Restaurant and is also quite central.<br/>
-His ideal neighborhood would be one which is central , doesn't have any Italian restaurant accessible to everyone and the number of restaurants would be less than 5<br/>

## Data:
-I have access to New York data which has the different neighborhoods and their longitude and latitude.<br/>
-First I will use this data with the FourSquare API to try to find the neighborhoods with Zero Italian Restaurants<br/>
-Then I will create a folium map of it and show it to the investor.<br/>


##  First Step
-Importing all necessary libraries <br/>

In [15]:
 #Importing necessary Libraries
    
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


##  Second Step
-Downloading and cleaing the data <br/>

In [16]:
#Get the Data
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')
    
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


In [17]:
#Cleaning the data
neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [19]:
#Filling the neighborhoods frame with the data imported

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
neighborhoods.head(4)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643


## Third Step:
-After I got the dataset and cleaned it above, Now i will go through all the Borough and find the Neighborhood with the minimum number of Italian Restaurants:<br/>
1-Access foursquare to get the food venues in each neighborhoods <br/>
2-Define a function that will : Get NearByFood places in all the neighborhoods in one Borough<br/>
3-Define a function that will extract the category of the venue<br/>

In [26]:
#ID required to access foursquare API 
CLIENT_ID = '1TITQDK2R10KCFRR3QVPRYQMHPKLTLYJZBXLJM1RP2IGRUVR' # your Foursquare ID
CLIENT_SECRET = 'LYDRUEYKILEJ04K55J5O2N2ZL4T4VV12HYVJPNZD4CMBW2KO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
search_query = 'food'
LIMIT=100



# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['Venue Category']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
#Get NearByFood places in all the neighborhoods in one Borough
def getNearbyVenues(names, latitudes, longitudes, radius=500):
        
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET,
            lat,
            lng,
            VERSION,
            search_query,
            radius,
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['venues']

         
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue Name', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    # filter the category for each row
    nearby_venues['Venue Category'] = nearby_venues.apply(get_category_type, axis=1)
    
    return(nearby_venues)


Now get the list of venues in all neighborhoods<br/>

In [27]:
#Get the list of all food venues in all neighborhoods
All_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

In [28]:
All_venues.head(6)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Baychester Avenue Food Truck,40.892293,-73.843230,Food Truck
1,Wakefield,40.894705,-73.847201,Taste the Island Bakery & Caribbean Food,40.899265,-73.846916,None
2,Wakefield,40.894705,-73.847201,Foodtown White Plains Rd,40.890900,-73.851403,Supermarket
3,Co-op City,40.874294,-73.829939,Food Universe Marketplace,40.876740,-73.828980,Grocery Store
4,Co-op City,40.874294,-73.829939,Food Universe Marketplace,40.870520,-73.828550,Supermarket
5,Co-op City,40.874294,-73.829939,Compare Foods,40.870647,-73.828425,Miscellaneous Shop


Define the function that will return the venues which have zero italian restaurants<br/>

In [29]:
#Define the function that will return the venues which have zero italian restaurants
def get_neighborhoods_zero_italian_restaurants(venues):
    # one hot encoding
    venues_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

    # add Neighborhood Latitude back to dataframe
    venues_onehot['Neighborhood Latitude'] = venues['Neighborhood Latitude'] 
    
    # add Neighborhood Longitude back to dataframe
    venues_onehot['Neighborhood Longitude'] = venues['Neighborhood Longitude']     
    
    # add neighborhood column back to dataframe
    venues_onehot['Neighborhood'] = venues['Neighborhood'] 
    
    # move neighborhood column to the first column
    fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])
    venues_onehot = venues_onehot[fixed_columns]

    #Group By Neighborhood
    venues_onehot=venues_onehot.groupby(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude']).sum()
    venues_onehot=venues_onehot.reset_index(drop = False)
    
    #Create a new Column for the total sum of Venues 
    col_list= list(venues_onehot)
    
    col_list.remove('Neighborhood')
    col_list.remove('Neighborhood Latitude')
    col_list.remove('Neighborhood Longitude')
    venues_onehot["TotalSum"]=venues_onehot[col_list].sum(axis=1)

    return (venues_onehot)



## Last Step:
-Get a list of all the neighborhoods who have zero italian restaurants:<br/>


In [31]:
#Get a list of all the neighborhoods who have 0 italian restaurnts 
neighborhood_zero_italian_restaurant=get_neighborhoods_zero_italian_restaurants(All_venues).reset_index(drop = True)

#Only selecting necessary columns
neighborhood_zero_italian_restaurant=neighborhood_zero_italian_restaurant[['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude','TotalSum']] 

numberOFRequiredRestaurants=5
#Selecting Neighborhoods with number of restaurants less than 5
neighborhood_zero_italian_restaurant=neighborhood_zero_italian_restaurant.loc[(neighborhood_zero_italian_restaurant['TotalSum'] <numberOFRequiredRestaurants)]

neighborhood_zero_italian_restaurant.head(5)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,TotalSum
2,Arrochar,40.596313,-74.067124,4
3,Arverne,40.589144,-73.791992,2
5,Astoria Heights,40.770317,-73.894680,4
10,Bay Terrace,40.553988,-74.139166,2
11,Bay Terrace,40.782843,-73.776802,2


## Visualisation:
-So now I have a list of all the Neighborhoods with zero italian restaurants, i will create a folium map for better demonstration.<br/>

In [32]:
#Address of New York City
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location= geolocator.geocode(address)
latitude_NewYork = location.latitude
longitude_NewYork = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude_NewYork, longitude_NewYork))

The geograpical coordinate of New York City are 40.7308619, -73.9871558.


In [33]:
# create map of New York using latitude and longitude of all the neighborhoods who have Zero Italian Restaurant
map_newyork = folium.Map(location=[latitude_NewYork, longitude_NewYork], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(neighborhood_zero_italian_restaurant['Neighborhood Latitude'], neighborhood_zero_italian_restaurant['Neighborhood Longitude'], neighborhood_zero_italian_restaurant['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [35]:
#Number of neighborhoods with zero italian restaurants in New York
neighborhood_zero_italian_restaurant.shape

(81, 4)

## Conclusion:
-There are 239 neighborhoods in New York with Zero italian restaurants .<br/>
-The investor could see those neighborhoods on the map and choose one of them for the location of his restaurant<br/>
